In [1]:
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [2]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

* 누락된 데이터를 다루는 가장 쉬운 방법 중 하나는 데이터셋에서 해당 샘플이나 특성을 완전히 삭제하는 것

In [3]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [4]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [5]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [6]:
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [7]:
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


* 편리하지만 너무 많은 데이터를 제거하면 안정된 분석이 불가능할 수 있다. 또한 클래스를 구분하는 데 필요한 중요한 정보를 잃을 위험도 있다.

## 누락된 값 대체

* 보간 기법 (interpololation) : 데이터셋에 있는 다른 훈련 샘플로부터 누락된 값을 추정. 가장 흔한 보간 기법 중 하나는 각 특성 열의 전체평균으로 대체.



In [14]:
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values = np.nan,strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

# 매개변수에 설정할 수 있는 다른값은 median(중앙값), most_frequent(최빈값), constant(지정값)가 있다
# 행 방향으로 계산하려면 FunctionTransformer 사용 

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

## 사이킷런 추정기 API 익히기
* 변환기(transformer) : 이런 추정기의 주요 메서드는 훈련 데이터에서 모델 파라미터를 학습하는 fit 메서드와 학습한 파라미터로 데이터를 변환하는 transform 메서드가 있다.
* 추정기(estimator) : svm, rf 등이 추정기에 속한다, predict 메서드가 있지만 transform 메서드도 가질 수 있다.

## 범주형 데이터 다루기

* 순서가 있는 특성과 순서가 없는 특성 : (ex. 티셔츠 사이즈 and 티셔츠 컬러)<br>
<center>(순서가 있다는 것은 비교 가능한 것)</center>

In [15]:
# 예제 데이터셋 만들기

import pandas as pd

df = pd.DataFrame([['green','M',10.1,'class1'],
                  ['red','L',13.5,'class2'],
                  ['blue','XL',15.3,'class1']])

df.columns = ['color','size','price','classlabel']
df

# 순서가 있는 size,price 특성

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


* 순서 특성 매핑 : 학습 알고리즘이 순서 특성을 올바르게 인식하려면 범주형의 문자열 값을 정수로 바꾸어야 한다. 즉, 위 데이터의 size를 예로 들면
$$ XL = L + 1 = M + 2 $$
이러한 매핑함수를 직접 만들어야함
* 순서를 가진 분류 or 회귀 : 사이킷런은 이에 대한 기능을 제공하지 않는다. 그러기 때문에 위와 같은 변환 과정을 거쳐야 한다. (ex. 영화 평점을 1~5까지 다섯 개의 순서가 있는 레이블로 예측하는 문제)

In [16]:
# 매핑함수

size_mapping = {'XL':3,
               'L':2,
               'M':1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [19]:
inv_size_mapping = {v:k for k,v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

* 클래스 레이블 인코딩 : 사이킷런의 분류 추정기 대부분은 자체적으로 클래스 레이블을 정수로 변환해 주지만 사소한 실수를 방지하기 위해서 클래스 레이블을 정수 배열로 전달하는 것이 좋은 습관이다.

In [21]:
import numpy as np
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [22]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [26]:
# 다른 방법
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y
# fit_transform 은 fit 과 transform 메서드를 합쳐놓은 단축 메서드

array([0, 1, 0], dtype=int64)

In [27]:
class_le.inverse_transform(y) # 안되는데..

array([0, 1, 0], dtype=int64)

* 순서가 없는 특성에 원-핫 인코딩 적용

In [29]:
X = df[['color','size','price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

위와 같이 특성을 변환하여 분류기에 주입하면  컬러 값에 어떤 순서가 없지만 학습 알고리즘이 red=2가 green=1보다 green=1이 blue=0 보다 크다고 가정하게 된다. 이 가정이 옳지 않지만 알고리즘이 의미 있는 결과를 만들 수 있고 이 결과는 최선이 아닐 것이다.
* 원-핫 인코딩 (one-hot encoding) : 위 문제를 해결하기 위한 통상적인 방법으로 아이디어는 순서 없는 특성에 들어 있는 고유한 값마다 새로운 더미 특성을 만드는 것이다.

In [38]:
# one-hot encoding

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ohe = OneHotEncoder(categories='auto')
col_trans = ColumnTransformer([('ohe',ohe,[0]),], remainder='passthrough')
col_trans.fit_transform(X)

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

In [35]:
# pandas를 이용한 다른 방법

pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


원-핫 인코딩된 데이터셋을 사용할 때 다중 공선성 문제를 유념해야한다. 특성 간의 상관관계가 높으면 역행렬을 계산하기 어려워 수치적으로 불안정해진다. 변수 간의 상관관계를 감소하려면 원-핫 이코딩된 배열에서 특성 열 하나를 삭제한다. 이렇게 특성을 삭제해도 잃는 정보는 없다.

In [36]:
pd.get_dummies(df[['price','color','size']],drop_first=True)

# green, red 둘다 0일 때 blue임을 알 수 있음

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


In [39]:
ohe = OneHotEncoder(categories='auto')
col_trans = ColumnTransformer([('ohe',ohe,[0]),], remainder='passthrough')
col_trans.fit_transform(X[1:])

array([[0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 3, 15.3]], dtype=object)